/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


/* Q12: Find the facilities with their usage by member, but not guests */


/* Q13: Find the facilities usage by month, but not guests */




In [5]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [6]:
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


In [109]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members! 
query1 = engine.execute(
    'SELECT finalquery.name, finalquery.revenue '+
    'FROM (SELECT MEMBER.name, (MEMBER.memberRevenue + GUEST.guestRevenue) AS revenue '+

           'FROM (SELECT name, (subq.facility_mem_amount * subq.membercost * subq.m_slots) AS memberRevenue '+
            'FROM (SELECT name, COUNT(*) AS facility_mem_amount, membercost, b.slots AS m_slots '+
                    'FROM Facilities AS f '+
                    'INNER JOIN Bookings AS b '+
                    'USING (facid) '+
                    'WHERE memid != 0 '+
                    'GROUP BY name) AS subq) AS MEMBER '+
            'INNER JOIN (SELECT name, (subq.facility_guest_amount * subq.guestcost * subq.slots) AS guestRevenue '+
                    'FROM (SELECT name, COUNT(*) AS facility_guest_amount, guestcost, b.slots AS slots '+
                    'FROM Facilities AS f '+
                    'INNER JOIN Bookings AS b '+
                    'USING (facid) '+
                    'WHERE memid = 0 '+
                    'GROUP BY name) AS subq) AS GUEST '+

            'USING (name)) AS finalquery '+
    'WHERE finalquery.revenue < 1000 '+
    'ORDER BY (finalquery.revenue)')


In [110]:
df = pd.DataFrame(query1.fetchall())
df.columns =query1.keys()
df

,name,revenue
0,Table Tennis,180
1,Pool Table,265
2,Snooker Table,460


In [96]:
# /* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */
query2 = engine.execute('SELECT m1.memid, m1.surname, m1.firstname, m1.address, m1.zipcode, m1.telephone, m1.joindate, m1.recommendedby, m2.surname AS recommended_by_sur, m2.firstname AS recommended_by_fir ' +
'FROM Members AS m1 '+
'INNER JOIN Members AS m2 '+
'ON m1.recommendedby = m2.memid '+
'WHERE m1.recommendedby != '"''"' '+
'ORDER BY m1.surname, m1.firstname ')


In [97]:
df_query2 = pd.DataFrame(query2.fetchall())
df_query2.columns = query2.keys()
df_query2

,memid,surname,firstname,address,zipcode,telephone,joindate,recommendedby,recommended_by_sur,recommended_by_fir
0,15,Bader,Florence,"264 Ursula Drive, Westford",84923,(833) 499-3527,2012-08-10 17:52:03,9,Stibbons,Ponder
1,12,Baker,Anne,"55 Powdery Street, Boston",80743,844-076-5141,2012-08-10 14:23:22,9,Stibbons,Ponder
2,16,Baker,Timothy,"329 James Street, Reading",58393,833-941-0824,2012-08-15 10:34:25,13,Farrell,Jemima
3,8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,2012-07-25 16:02:35,3,Rownam,Tim
4,5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,2012-07-09 10:44:09,1,Smith,Darren
5,22,Coplin,Joan,"85 Bard Street, Bloomington, Boston",43533,(822) 499-2232,2012-08-29 08:32:41,16,Baker,Timothy
6,36,Crumpet,Erica,"Crimson Road, North Reading",75655,(811) 732-4816,2012-09-22 08:36:38,2,Smith,Tracy
7,7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,2012-07-25 08:59:12,4,Joplette,Janice
8,20,Genting,Matthew,"4 Nunnington Place, Wingfield, Boston",52365,(811) 972-1377,2012-08-19 14:55:55,5,Butters,Gerald
9,35,Hunt,John,"5 Bullington Lane, Boston",54333,(899) 720-6978,2012-09-19 11:32:45,30,Purview,Millicent


In [127]:
query3 = engine.execute('SELECT f.name, b.memid ' +
                        'FROM Facilities  AS f '+
                        'INNER JOIN Bookings AS b '+
                        'USING (facid) '+
                        'WHERE b.memid != 0 '+
                        'GROUP BY f.name, b.memid '+
                        'ORDER BY b.memid')

In [128]:
df_query3 = pd.DataFrame(query3.fetchall())
df_query3.columns = query3.keys()
df_query3

,name,memid
0,Badminton Court,1
1,Massage Room 1,1
2,Pool Table,1
3,Snooker Table,1
4,Squash Court,1
...,...,...
197,Tennis Court 2,35
198,Badminton Court,36
199,Massage Room 1,36
200,Table Tennis,36


In [133]:
query4 = engine.execute('SELECT f.name, b.memid '+
                        'FROM Facilities AS f '+
                        'INNER JOIN Bookings AS b '+
                        'USING (facid) '+
                        'WHERE b.memid != 0 '+
                        'GROUP BY f.name, strftime(\'%m\', date(b.starttime)) '+
                        'ORDER BY f.name, strftime(\'%m\', date(b.starttime))')

In [134]:
df_query4 = pd.DataFrame(query4.fetchall())
df_query4.columns = query4.keys()
df_query4

,name,memid
0,Badminton Court,1
1,Badminton Court,1
2,Badminton Court,1
3,Massage Room 1,1
4,Massage Room 1,5
5,Massage Room 1,8
6,Massage Room 2,4
7,Massage Room 2,7
8,Massage Room 2,7
9,Pool Table,1
